In [1]:
import numpy as np
import pandas as pd
from src.describe import summary_cluster, one_hot_encode_clusters

設定cluster csv路徑並讀取資料

In [2]:
cluster_path = './cluster-result/1713836452_som/cluster_130.csv'
cluster = pd.read_csv(cluster_path)

根據cluster進行劃分

In [3]:

chart = summary_cluster(cluster)
chart.render_notebook()

# Unsupervised to Supervised

Convert each cluster to binary


In [7]:
one_hot_encoded_clusters = one_hot_encode_clusters(cluster)

one_hot_encoded_cluster = one_hot_encoded_clusters[0]

X = one_hot_encoded_cluster[selected_features]  # 選取特定的欄位作為特徵
y = one_hot_encoded_cluster['target_column']

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x4_normalized,x5_normalized,x6_normalized,x7_normalized,x8_normalized,x9_normalized,x10_normalized,x11_normalized,x12_normalized,cluster
0,0,0.375285,0.001805,0.339836,0.307452,0.270558,0.573429,0.376695,0.271439,0.177600,...,0.307452,0.270558,0.573429,0.376695,0.271439,0.177600,-0.000007,0.000013,-0.000011,1
1,1,0.382663,0.001826,0.345361,0.312627,0.276324,0.554112,0.378665,0.278288,0.179802,...,0.312627,0.276324,0.554112,0.378665,0.278288,0.179802,-0.000007,0.000012,-0.000010,0
2,2,0.382313,0.001824,0.346351,0.311250,0.273926,0.567248,0.368509,0.267866,0.180669,...,0.311250,0.273926,0.567248,0.368509,0.267866,0.180669,-0.000008,0.000013,-0.000010,1
3,3,0.385215,0.001826,0.347327,0.312501,0.275298,0.563200,0.367964,0.268203,0.181659,...,0.312501,0.275298,0.563200,0.367964,0.268203,0.181659,-0.000008,0.000012,-0.000010,1
4,4,0.379658,0.001784,0.345031,0.311953,0.277820,0.554760,0.379235,0.281655,0.177213,...,0.311953,0.277820,0.554760,0.379235,0.281655,0.177213,-0.000007,0.000012,-0.000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,0.357593,0.001735,0.325239,0.301754,0.279066,0.552982,0.408284,0.311121,0.167718,...,0.301754,0.279066,0.552982,0.408284,0.311121,0.167718,-0.000005,0.000011,-0.000010,0
249996,249996,0.353033,0.001685,0.325741,0.303431,0.291955,0.493385,0.442432,0.355572,0.162308,...,0.303431,0.291955,0.493385,0.442432,0.355572,0.162308,-0.000002,0.000008,-0.000008,0
249997,249997,0.347164,0.001649,0.321791,0.302658,0.297047,0.491559,0.446895,0.359867,0.158862,...,0.302658,0.297047,0.491559,0.446895,0.359867,0.158862,-0.000002,0.000008,-0.000007,0
249998,249998,0.357483,0.001743,0.323123,0.299783,0.270499,0.554555,0.414648,0.311470,0.168156,...,0.299783,0.270499,0.554555,0.414648,0.311470,0.168156,-0.000005,0.000011,-0.000010,0


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
